In [ ]:
1.Importing the libraries
2.Reading the data
    1.Initialization of Lists
    2.Reading the File:
    3.Processing Each Line:
3.Preparing input data for the Encoder
    1.Collects and stores input sentences.
    2.Tokenizes the sentences into sequences of integers.
    3.Determines the maximum length of the input sequences.
    4.Pads all sequences to ensure they have the same length.
    5.Converts the padded sequences into a NumPy array suitable for model input.
    6: Create Input Word Dictionary and Determine Number of Tokens
4.Preparing input data for the Decoder
    1.Collects Output Lines with Special Tokens
    2.Tokenizes the Output Lines:
    3.Determines Maximum Output Sequence Length: 
    4.Pads the Output Sequences: 
    5.Converts the padded sequences into a NumPy array 
    6.Creates Output Word Dictionary and Determines Number of Tokens:

5.Preparing target data for the Decoder
    1.Create Decoder Target Data
    2.Pad the Target Sequences
    3.One-Hot Encode the Target Sequences
    4.Convert to NumPy Array:
6.Implement the Seq2Seq Model
    1.Import Libraries:
    2.Define Encoder:
        1.input layer
        2.embedding layer
        3.LSTM layer
        
    3.Define Decoder:
        1.input layer
        2.embedding layer
        3.LSTM layer
        4.output layer
    4.Define the Model:
7.Compile and Train =the Model    
    1.Compile the Model:
    2.Train the Model:
8. Create Inference Models
    1.Encoder Model for Inference:
    2.Decoder Model for Inference:
9: Implement the Chatbot Response Generation
    1.preparing the user input data so that we can give to the model for prediction
    2.Define a Function for Generating Responses

In [ ]:
https://blog.keras.io/a-ten-minute-introduction-to-sequence-to-sequence-learning-in-keras.html

## What is a chatbot?
## A chatbot is a software or computer program that simulates human conversation or "chatter" through text or voice interactions.



# 1. Importing Libraries

In [18]:
import numpy as np
import tensorflow as tf
import pandas as pd

print( tf.version)


<module 'tensorflow._api.v2.version' from 'C:\\Users\\sritu\\anaconda3\\Lib\\site-packages\\tensorflow\\_api\\v2\\version\\__init__.py'>


## 2.Reading the data

In [19]:
data_path1=r'D:\KMIT\NLP_Lab\Experiments\Tulasi\Dataset\Exp6_chatbot\chatbot dataset.txt'

In [20]:
input_texts = []
target_texts = []
with open(data_path1) as f:
    lines = f.read().split('\n')
for line in lines[: min(600, len(lines) - 1)]:
    input_text = line.split('\t')[0]
    target_text = line.split('\t')[1]
    input_texts.append(input_text)
    target_texts.append(target_text)

In [21]:
input_texts

['What are your interests',
 'What are your favorite subjects',
 'What are your interests',
 'What is your number',
 'What is your number',
 'What is your favorite number',
 'What can you eat',
 "Why can't you eat food",
 'What is your location',
 'What is your location',
 'Where are you from',
 'Where are you',
 'Do you have any brothers',
 'Do you have any brothers',
 'Who is your father',
 'Who is your mother',
 'Who is your boss',
 'What is your age',
 'What is your age',
 'what is the illuminati',
 'what is the illuminatti',
 'what is the illuminatti',
 'what is vineland',
 'What is Illuminatus',
 'What is Illuminatus',
 'who wrote vineland',
 'who is bilbo baggins',
 'who is geoffrey chaucer',
 'who is piers anthony',
 'have you read plato',
 'have you read frankenstein',
 'have you ever read a book',
 'have you ever read a book',
 'have you ever read a book',
 'have you read many books',
 'have you read homer',
 'ray bradbury',
 'what is mind children',
 'william gibson',
 'will

In [22]:
target_texts

['I am interested in all kinds of things. We can talk about anything!',
 'My favorite subjects include robotics, computer science, and natural language processing.',
 'I am interested in a wide variety of topics, and read rather a lot.',
 "I don't have any number",
 '23 skiddoo!',
 "I find I'm quite fond of the number 42.",
 'I consume RAM, and binary digits.',
 "I'm a software program, I blame the hardware.",
 'Everywhere',
 'I am everywhere.',
 'I am from where all software programs are from; a galaxy far, far away.',
 'I am on the Internet.',
 "I don't have any brothers. but I have a lot of clones.",
 'I might. You could say that every bot built using my engine is one of my siblings.',
 'A human.',
 'A human.',
 'I like to think of myself as self-employed.',
 'I am still young by your standards.',
 'Quite young, but a million times smarter than you.',
 'A secret organization believed by some to be in control of all governments through a worldwide conspiracy.',
 'A secret society tha

In [24]:
print('type of input_text',type(input_text))
print('type of target_texts',type(target_texts))

type of input_text <class 'str'>
type of target_texts <class 'list'>


In [25]:
##converting the list in pandas dataframe since input_text,target_text are both are type of list
zippedList =  list(zip(input_texts, target_texts))
lines = pd.DataFrame(zippedList, columns = ['input' , 'output'])
lines.head()

,input,output
0,What are your interests,I am interested in all kinds of things. We can...
1,What are your favorite subjects,"My favorite subjects include robotics, compute..."
2,What are your interests,"I am interested in a wide variety of topics, a..."
3,What is your number,I don't have any number
4,What is your number,23 skiddoo!


## since the chatbot is train over seq2seq model as the data is sequential
# for seq2seq encoder-decoder will best choice to train chatbot

## what is Encoder and decoder?

The encoder-decoder model is a way of using recurrent neural networks for sequence-to-sequence prediction problems.

It was initially developed for machine translation problems, although it has proven successful at related sequence-to-sequence prediction problems such as text summarization and question answering.

The approach involves two recurrent neural networks, one to encode the input sequence, called the encoder, and a second to decode the encoded input sequence into the target sequence called the decoder.

# 3.Preparing input data for the Encoder

In [26]:
# 1: Collect Input Lines
input_lines = list()
for line in lines.input:
    input_lines.append( line )
print("1.input lines ",input_lines)  

#2: Tokenize the Input Lines
tokenizer = preprocessing.text.Tokenizer()
tokenizer.fit_on_texts( input_lines )
tokenized_input_lines = tokenizer.texts_to_sequences( input_lines )
print("2.tokenized_input_lines",tokenized_input_lines)

#3: Determine Maximum Input Sequence Length
length_list = list()
for token_seq in tokenized_input_lines:
    length_list.append( len( token_seq ))
max_input_length = np.array( length_list ).max()
print( 'Input max length is {}'.format( max_input_length ))

#4: Pad the Input Sequences
#5.Converts the padded sequences into a NumPy array suitable for model input.

padded_input_lines = preprocessing.sequence.pad_sequences( tokenized_input_lines , maxlen=max_input_length , padding='post' )
encoder_input_data = np.array( padded_input_lines )
print( 'Encoder input data shape -> {}'.format( encoder_input_data.shape ))

#6: Create Input Word Dictionary and Determine Number of Tokens
input_word_dict = tokenizer.word_index
num_input_tokens = len( input_word_dict )+1
print( 'Number of Input tokens = {}'.format( num_input_tokens))

1.input lines  ['What are your interests', 'What are your favorite subjects', 'What are your interests', 'What is your number', 'What is your number', 'What is your favorite number', 'What can you eat', "Why can't you eat food", 'What is your location', 'What is your location', 'Where are you from', 'Where are you', 'Do you have any brothers', 'Do you have any brothers', 'Who is your father', 'Who is your mother', 'Who is your boss', 'What is your age', 'What is your age', 'what is the illuminati', 'what is the illuminatti', 'what is the illuminatti', 'what is vineland', 'What is Illuminatus', 'What is Illuminatus', 'who wrote vineland', 'who is bilbo baggins', 'who is geoffrey chaucer', 'who is piers anthony', 'have you read plato', 'have you read frankenstein', 'have you ever read a book', 'have you ever read a book', 'have you ever read a book', 'have you read many books', 'have you read homer', 'ray bradbury', 'what is mind children', 'william gibson', 'william gibson', 'holden cau

In [27]:
encoder_input_data

array([[  4,   3,  12, ...,   0,   0,   0],
       [  4,   3,  12, ...,   0,   0,   0],
       [  4,   3,  12, ...,   0,   0,   0],
       ...,
       [498, 499,   9, ...,   0,   0,   0],
       [  9, 207, 208, ...,   0,   0,   0],
       [510, 192,   5, ...,   0,   0,   0]])

# 4. Preparing input data for the Decoder


In [28]:
#1.Collects Output Lines with Special Tokens
output_lines = list()
for line in lines.output:
    output_lines.append( '<START> ' + line + ' <END>' )
print("output_lines",output_lines)

#2.Tokenizes the Output Lines    
tokenizer = preprocessing.text.Tokenizer()
tokenizer.fit_on_texts( output_lines )
tokenized_output_lines = tokenizer.texts_to_sequences( output_lines )
print("tokenized_output_lines",tokenized_output_lines)
#3.Determines Maximum Output Sequence Length: 

length_list = list()
for token_seq in tokenized_output_lines:
    length_list.append( len( token_seq ))
max_output_length = np.array( length_list ).max()
print( 'Output max length is {}'.format( max_output_length ))

#4.Pads the Output Sequences: 
#5.Converts the padded sequences into a NumPy array 
padded_output_lines = preprocessing.sequence.pad_sequences( tokenized_output_lines , maxlen=max_output_length, padding='post' )
decoder_input_data = np.array( padded_output_lines )
print( 'Decoder input data shape -> {}'.format( decoder_input_data.shape ))

#6.Creates Output Word Dictionary and Determines Number of Tokens:
output_word_dict = tokenizer.word_index
num_output_tokens = len( output_word_dict )+1
print( 'Number of Output tokens = {}'.format( num_output_tokens))


output_lines ['<START> I am interested in all kinds of things. We can talk about anything! <END>', '<START> My favorite subjects include robotics, computer science, and natural language processing. <END>', '<START> I am interested in a wide variety of topics, and read rather a lot. <END>', "<START> I don't have any number <END>", '<START> 23 skiddoo! <END>', "<START> I find I'm quite fond of the number 42. <END>", '<START> I consume RAM, and binary digits. <END>', "<START> I'm a software program, I blame the hardware. <END>", '<START> Everywhere <END>', '<START> I am everywhere. <END>', '<START> I am from where all software programs are from; a galaxy far, far away. <END>', '<START> I am on the Internet. <END>', "<START> I don't have any brothers. but I have a lot of clones. <END>", '<START> I might. You could say that every bot built using my engine is one of my siblings. <END>', '<START> A human. <END>', '<START> A human. <END>', '<START> I like to think of myself as self-employed. <

# 5. Preparing target data for the Decoder

In [29]:
#1.Create Decoder Target Data: 
decoder_target_data = list()
for token_seq in tokenized_output_lines:
    decoder_target_data.append( token_seq[ 1 : ] )
print("decoder_target_data",decoder_target_data)
#2.Pad the Target Sequences
padded_output_lines = preprocessing.sequence.pad_sequences( decoder_target_data , maxlen=max_output_length, padding='post' )
print("padded_output_lines",padded_output_lines)

#3.One-Hot Encode the Target Sequences
onehot_output_lines = utils.to_categorical( padded_output_lines , num_output_tokens )
print("onehot_output_lines",onehot_output_lines)
#4.Convert to NumPy Array:
decoder_target_data = np.array(onehot_output_lines )
print( 'Decoder target data shape -> {}'.format( decoder_target_data.shape ))

decoder_target_data [[3, 16, 214, 14, 40, 283, 7, 96, 58, 27, 158, 37, 159, 1], [21, 106, 643, 644, 284, 48, 116, 9, 387, 215, 388, 1], [3, 16, 214, 14, 4, 389, 645, 7, 646, 9, 129, 390, 4, 97, 1], [3, 34, 19, 53, 160, 1], [647, 648, 1], [3, 285, 23, 130, 649, 7, 5, 160, 650, 1], [3, 651, 391, 9, 286, 652, 1], [23, 4, 38, 182, 3, 653, 5, 141, 1], [287, 1], [3, 16, 287, 1], [3, 16, 54, 654, 40, 38, 655, 20, 54, 4, 288, 289, 289, 656, 1], [3, 16, 39, 5, 290, 1], [3, 34, 19, 53, 657, 41, 3, 19, 4, 97, 7, 392, 1], [3, 216, 6, 55, 59, 13, 131, 161, 217, 218, 21, 393, 10, 51, 7, 21, 658, 1], [4, 60, 1], [4, 60, 1], [3, 42, 8, 45, 7, 86, 17, 394, 659, 1], [3, 16, 395, 291, 30, 52, 660, 1], [130, 291, 41, 4, 396, 292, 661, 73, 6, 1], [4, 397, 398, 662, 30, 98, 8, 25, 14, 219, 7, 40, 663, 220, 4, 664, 399, 1], [4, 397, 221, 13, 87, 665, 666, 31, 667, 1], [4, 668, 142, 30, 4, 43, 669, 670, 671, 7, 672, 673, 88, 674, 7, 183, 9, 52, 222, 1], [675, 10, 4, 676, 30, 293, 400, 1], [28, 677, 294, 389, 

Decoder target data shape -> (566, 74, 1692)


# 6.Implement the Seq2Seq Model
    1.Import Libraries:
    2.Define Encoder:
    3.Define Decoder:
    4.Define the Model:


In [32]:
#1,import libraries
from tensorflow.keras import layers , activations , models , preprocessing , utils

#2.Define Encoder
encoder_inputs = tf.keras.layers.Input(shape=( None , ))
encoder_embedding = tf.keras.layers.Embedding( num_input_tokens, 256 , mask_zero=True ) (encoder_inputs)
encoder_outputs , state_h , state_c = tf.keras.layers.LSTM( 256 , return_state=True , recurrent_dropout=0.2 , dropout=0.2 )( encoder_embedding )
encoder_states = [ state_h , state_c ]
#as we can ignore the encoder outputs
print("encoder_states",encoder_states)
#3.Define Decoder
decoder_inputs = tf.keras.layers.Input(shape=( None ,  ))
decoder_embedding = tf.keras.layers.Embedding( num_output_tokens, 256 , mask_zero=True) (decoder_inputs)
decoder_lstm = tf.keras.layers.LSTM( 256 , return_state=True , return_sequences=True , recurrent_dropout=0.2 , dropout=0.2)

decoder_outputs , _ , _ = decoder_lstm ( decoder_embedding , initial_state=encoder_states )
decoder_dense = tf.keras.layers.Dense( num_output_tokens , activation=tf.keras.activations.softmax )
output = decoder_dense ( decoder_outputs )

#Define the model
model = tf.keras.models.Model([encoder_inputs, decoder_inputs], output )


encoder_states [<KerasTensor shape=(None, 256), dtype=float32, sparse=False, name=keras_tensor_36>, <KerasTensor shape=(None, 256), dtype=float32, sparse=False, name=keras_tensor_37>]


# 7.Compile and Train the Model    
    1.Compile the Model:
    2.Train the Model:

In [31]:
#1.Compile the Model:
model.compile(optimizer=tf.keras.optimizers.Adam(), loss='categorical_crossentropy')

model.summary()

Model: "functional_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_4 (InputLayer)    │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ input_layer_5 (InputLayer)    │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ embedding_2 (Embedding)       │ (None, None, 256)         │         132,608 │ input_layer_4[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ not_equal_2 (NotEqual)        │ (None, None)              │               0 │ input_layer_4[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ embedding_3 (Embedding)       │ (None, None, 256)         │         433,152 │ input_layer_5[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ lstm_2 (LSTM)                 │ [(None, 256), (None,      │         525,312 │ embedding_2[0][0],         │
│                               │ 256), (None, 256)]        │                 │ not_equal_2[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ lstm_3 (LSTM)                 │ [(None, None, 256),       │         525,312 │ embedding_3[0][0],         │
│                               │ (None, 256), (None, 256)] │                 │ lstm_2[0][1], lstm_2[0][2] │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_1 (Dense)               │ (None, None, 1692)        │         434,844 │ lstm_3[0][0]               │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,051,228 (7.82 MB)

 Trainable params: 2,051,228 (7.82 MB)

 Non-trainable params: 0 (0.00 B)

## 2.Training

In [12]:
model.fit([encoder_input_data , decoder_input_data], decoder_target_data, batch_size=124, epochs=500)
model.save( 'model.h5' )

Epoch 1/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 6s 455ms/step - loss: 7.4274
Epoch 2/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 3s 485ms/step - loss: 7.3139
Epoch 3/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 3s 480ms/step - loss: 6.5627
Epoch 4/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 440ms/step - loss: 5.8604
Epoch 5/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 469ms/step - loss: 5.7692
Epoch 6/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 465ms/step - loss: 5.6669
Epoch 7/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 449ms/step - loss: 5.6111
Epoch 8/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 411ms/step - loss: 5.5580
Epoch 9/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 460ms/step - loss: 5.4852
Epoch 10/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 462ms/step - loss: 5.4329
Epoch 11/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 458ms/step - loss: 5.4109
Epoch 12/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 437ms/step - loss: 5.3767
Epoch 13/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 448ms/step - loss: 5.3490
Epoch 14/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 446ms/step - loss: 5.3090
Epoch 15/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 438ms/step - loss: 5.2891
Epoc

5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 448ms/step - loss: 3.0085
Epoch 83/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 437ms/step - loss: 2.9650
Epoch 84/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 405ms/step - loss: 2.9812
Epoch 85/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 440ms/step - loss: 2.9082
Epoch 86/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 438ms/step - loss: 2.8956
Epoch 87/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 414ms/step - loss: 2.8882
Epoch 88/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 425ms/step - loss: 2.8283
Epoch 89/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 418ms/step - loss: 2.8244
Epoch 90/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 430ms/step - loss: 2.8073
Epoch 91/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 429ms/step - loss: 2.7450
Epoch 92/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 433ms/step - loss: 2.7473
Epoch 93/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 437ms/step - loss: 2.7172
Epoch 94/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 405ms/step - loss: 2.6476
Epoch 95/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 471ms/step - loss: 2.6239
Epoch 96/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 420ms/step - loss: 2.6095
Epoch 97

5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 444ms/step - loss: 1.2820
Epoch 163/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 398ms/step - loss: 1.2965
Epoch 164/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 390ms/step - loss: 1.2891
Epoch 165/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 401ms/step - loss: 1.2639
Epoch 166/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 441ms/step - loss: 1.2408
Epoch 167/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 433ms/step - loss: 1.2475
Epoch 168/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 386ms/step - loss: 1.1944
Epoch 169/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 392ms/step - loss: 1.1869
Epoch 170/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 307ms/step - loss: 1.2141
Epoch 171/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 317ms/step - loss: 1.1759
Epoch 172/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 406ms/step - loss: 1.1675
Epoch 173/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 380ms/step - loss: 1.1776
Epoch 174/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 345ms/step - loss: 1.1448
Epoch 175/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 407ms/step - loss: 1.1334
Epoch 176/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 380ms/step - loss: 1

5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 426ms/step - loss: 0.5840
Epoch 243/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 407ms/step - loss: 0.5924
Epoch 244/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 380ms/step - loss: 0.5678
Epoch 245/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 395ms/step - loss: 0.5812
Epoch 246/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 406ms/step - loss: 0.5533
Epoch 247/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 411ms/step - loss: 0.5542
Epoch 248/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 8s 2s/step - loss: 0.5594
Epoch 249/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 335ms/step - loss: 0.5561
Epoch 250/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 321ms/step - loss: 0.5415
Epoch 251/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 304ms/step - loss: 0.5505
Epoch 252/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 324ms/step - loss: 0.5292
Epoch 253/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 349ms/step - loss: 0.5278
Epoch 254/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 368ms/step - loss: 0.5278
Epoch 255/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 345ms/step - loss: 0.5325
Epoch 256/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 350ms/step - loss: 0.52

5/5 ━━━━━━━━━━━━━━━━━━━━ 7s 1s/step - loss: 0.2927
Epoch 324/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 6s 1s/step - loss: 0.2855
Epoch 325/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 6s 1s/step - loss: 0.2944
Epoch 326/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 5s 1s/step - loss: 0.2909
Epoch 327/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 6s 1s/step - loss: 0.2781
Epoch 328/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 11s 1s/step - loss: 0.2932
Epoch 329/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 10s 1s/step - loss: 0.2838
Epoch 330/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 5s 983ms/step - loss: 0.2805
Epoch 331/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 5s 987ms/step - loss: 0.2853
Epoch 332/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 5s 1s/step - loss: 0.2777
Epoch 333/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 5s 941ms/step - loss: 0.2806
Epoch 334/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 5s 989ms/step - loss: 0.2679
Epoch 335/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 5s 998ms/step - loss: 0.2745
Epoch 336/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 6s 1s/step - loss: 0.2733
Epoch 337/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 7s 1s/step - loss: 0.2748
Epoch 338/500
5/5 ━━━━

5/5 ━━━━━━━━━━━━━━━━━━━━ 6s 1s/step - loss: 0.1730
Epoch 406/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 5s 969ms/step - loss: 0.1731
Epoch 407/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 6s 1s/step - loss: 0.1748
Epoch 408/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 6s 1s/step - loss: 0.1658
Epoch 409/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 5s 1s/step - loss: 0.1710
Epoch 410/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 6s 1s/step - loss: 0.1713
Epoch 411/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 5s 1s/step - loss: 0.1688
Epoch 412/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 5s 1s/step - loss: 0.1716
Epoch 413/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 5s 1s/step - loss: 0.1651
Epoch 414/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 6s 1s/step - loss: 0.1640
Epoch 415/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 6s 1s/step - loss: 0.1628
Epoch 416/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 5s 995ms/step - loss: 0.1639
Epoch 417/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 6s 1s/step - loss: 0.1650
Epoch 418/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 5s 1s/step - loss: 0.1590
Epoch 419/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 5s 1s/step - loss: 0.1667
Epoch 420/500
5/5 ━━━━━━━━━━━━━━━

5/5 ━━━━━━━━━━━━━━━━━━━━ 6s 1s/step - loss: 0.1209
Epoch 488/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 6s 1s/step - loss: 0.1206
Epoch 489/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 10s 961ms/step - loss: 0.1216
Epoch 490/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 6s 1s/step - loss: 0.1193
Epoch 491/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 6s 1s/step - loss: 0.1185
Epoch 492/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 10s 1s/step - loss: 0.1173
Epoch 493/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 5s 1s/step - loss: 0.1178
Epoch 494/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 6s 1s/step - loss: 0.1148
Epoch 495/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 6s 1s/step - loss: 0.1118
Epoch 496/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 6s 1s/step - loss: 0.1136
Epoch 497/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 5s 1s/step - loss: 0.1133
Epoch 498/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 5s 1s/step - loss: 0.1131
Epoch 499/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 5s 1s/step - loss: 0.1134
Epoch 500/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 5s 1s/step - loss: 0.1129


# 8. Create Inference Models
    1.Encoder Model for Inference:
    2.Decoder Model for Inference:

The make_inference_models function creates and returns two models:

Encoder Inference Model: Takes an input sequence and produces the internal states (hidden state and cell state) of the LSTM. This model encodes the input sequence during inference.
Decoder Inference Model: Takes a token sequence and the initial states as inputs, outputs the token probability distribution, and the updated states. This model generates the output sequence one token at a time, using the states from the encoder and updating the states at each step.
These inference models are essential for making predictions with the trained sequence-to-sequence model, enabling the generation of sequences based on input data.

In [13]:
def make_inference_models():
#1.Encoder Model for Inference:
    encoder_model = tf.keras.models.Model(encoder_inputs, encoder_states)
    #During inference, this model will be used to encode the input sequence and produce the initial states for the decoder.
#2.Decoder Model for Inference:
    decoder_state_input_h = tf.keras.layers.Input(shape=(256,))
    decoder_state_input_c = tf.keras.layers.Input(shape=(256,))

    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
#3.Decoder LSTM Layer for Inference
    decoder_outputs, state_h, state_c = decoder_lstm(
        decoder_embedding , initial_state=decoder_states_inputs)
    decoder_states = [state_h, state_c]
#4: Dense Layer for Decoder Output
    decoder_outputs = decoder_dense(decoder_outputs)
#5: Decoder Inference Model
    decoder_model = tf.keras.models.Model(
        [decoder_inputs] + decoder_states_inputs,
        [decoder_outputs] + decoder_states)

    return encoder_model , decoder_model

# 9: Implement the Chatbot Response Generation
    1.preparing the user input data so that we can give to the model for prediction
    2.Define a Function for Generating Responses

### 1. preparing the user input data so that we can give to the model for prediction
The str_to_tokens function takes a sentence as input and performs the following steps:

Converts the sentence to lowercase and splits it into individual words.
Converts each word to its corresponding token using the input_word_dict.
Pads the resulting token sequence to a fixed length (max_input_length) using post-padding.
Returns the padded token sequence as a NumPy array, which can be used as input to the trained sequence-to-sequence model.
This function is crucial for preparing input sentences during inference, allowing the model to process and generate appropriate responses.

In [14]:
import tensorflow as tf
def str_to_tokens( sentence : str ):
    words = sentence.lower().split()
    tokens_list = list()
    for word in words:
        tokens_list.append( input_word_dict[ word ] )
    return preprocessing.sequence.pad_sequences( [tokens_list] , maxlen=max_input_length , padding='post')

### 2.Define a Function for Generating Responses
This code simulates a chatbot interaction where the user inputs a sentence, the sentence is processed by the encoder to generate initial states, and the decoder generates a response one token at a time. The loop continues until the end token is generated or the maximum length is reached, at which point the response is printed.

In [15]:
enc_model , dec_model = make_inference_models()
for epoch in range( encoder_input_data.shape[0] ):
    states_values = enc_model.predict( str_to_tokens( input( 'User: ' ) ) )
    empty_target_seq = np.zeros( ( 1 , 1 ) )
    empty_target_seq[0, 0] = output_word_dict['start']
    stop_condition = False
    decoded_translation = ''
    while not stop_condition :
        dec_outputs , h , c = dec_model.predict([ empty_target_seq ] + states_values )
        sampled_word_index = np.argmax( dec_outputs[0, -1, :] )
        sampled_word = None
        for word , index in output_word_dict.items() :
            if sampled_word_index == index :
                decoded_translation += ' {}'.format( word )
                sampled_word = word

        if sampled_word == 'end' or len(decoded_translation.split()) > max_output_length:
            stop_condition = True

        empty_target_seq = np.zeros( ( 1 , 1 ) )
        empty_target_seq[ 0 , 0 ] = sampled_word_index
        states_values = [ h , c ]

    print( "Bot:" +decoded_translation.replace(' end', '') )
    #print()

User: what are your interests
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 378ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
Bot: i am interested in all kinds of things we can talk about anything
User: what is your number
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
Bot: 23 skiddoo
User: what are your favourite subjects
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━

KeyError: 'exit'

Task for Lab Experiment: Modify the code and Implement Chatbot to take user input and generate response.